In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt


import os
import json

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

path_dir = 'D:/data/timeline'
path_dir_match = 'D:/data/match'


In [3]:
X = []
y = []
file_list = os.listdir(path_dir)
file_list_match = os.listdir(path_dir_match)

union = list(set(file_list) & set(file_list_match))
print(len(file_list))
print(len(file_list_match))
print(len(union))

34287
34287
34287


In [4]:

def get_features(parcipnt, time):
    features = {'kills':[], 'deaths':[], 'assists':[], 'largestKillingSpree':[], 'longestTimeSpentLiving':[], 'totalDamageDealt':[], 
            'magicDamageDealt':[], 'physicalDamageDealt':[], 'trueDamageDealt':[], 'totalDamageDealtToChampions':[], 
            'magicDamageDealtToChampions':[], 'physicalDamageDealtToChampions':[], 'trueDamageDealtToChampions':[],
            'totalHeal':[], 'totalUnitsHealed':[], 'totalDamageTaken':[], 'goldEarned':[],
            'totalMinionsKilled':[], 'timeCCingOthers':[], 'wardsPlaced':[], 'wardsKilled':[]}

    ward_place = False
    ward_killed = False

    for col in parcipnt:
        try:
            if col == 'win':
                labels['win']=participant[0][col]
            else:
                if col in features:
                    if col == 'wardsPlaced':
                        ward_place = True
                    elif col == 'wardsKilled':
                        ward_killed = True
                    features[col]=parcipnt[col]/time

        except:
            continue

    if not ward_place:
        features['wardsPlaced']=0
    if not ward_killed:
        features['wardsKilled']=0
        
    return features

def get_graph(file_name, file_name_match):
    
    with open(file_name, 'rt', encoding='UTF8') as f:
        json_data = json.load(f)
        data = json.dumps(json_data)
        
    with open(file_name_match, 'rt', encoding='UTF8') as f:
        match_json_data = json.load(f)
        match_data = json.dumps(match_json_data)

    #print(json_data["metadata"])
    participants = match_json_data["metadata"]["participants"]
    if match_json_data["info"]["teams"][0]["win"]:
        win_team = 0
    else:
        win_team = 1
        
    line_part = [[] for i in range(10)]
    id_line_match = {}
    
    for parcipnt in match_json_data["info"]["participants"]:
        win = 0
        if parcipnt["teamId"] == 200:
            win = 5
        if parcipnt["teamPosition"] == "TOP":
            for p in json_data["info"]["participants"]:
                if p["puuid"] == parcipnt["puuid"]:
                    id_line_match[str(p["participantId"])] = 0+win
                    
        elif parcipnt["teamPosition"] == "JUNGLE":
            for p in json_data["info"]["participants"]:
                if p["puuid"] == parcipnt["puuid"]:
                    id_line_match[str(p["participantId"])] = 1+win
                    
        elif parcipnt["teamPosition"] == "MIDDLE":
            for p in json_data["info"]["participants"]:
                if p["puuid"] == parcipnt["puuid"]:
                    id_line_match[str(p["participantId"])] = 2+win
                    
        elif parcipnt["teamPosition"] == "BOTTOM":
            for p in json_data["info"]["participants"]:
                if p["puuid"] == parcipnt["puuid"]:
                    id_line_match[str(p["participantId"])] = 3+win
                    
        elif parcipnt["teamPosition"] == "UTILITY":
            for p in json_data["info"]["participants"]:
                if p["puuid"] == parcipnt["puuid"]:
                    id_line_match[str(p["participantId"])] = 4+win
    
    if len(id_line_match.keys())!=10:
        print(id_line_match)
        return np.array([]), np.array([])
    
    frame = json_data["info"]["frames"][0]
    
    for i in range(1, 11):
        data = np.array([list(frame["participantFrames"][str(i)]["championStats"].values()) + list(frame["participantFrames"][str(i)]["damageStats"].values()) + [frame["participantFrames"][str(i)]["currentGold"]] + [frame["participantFrames"][str(i)]["goldPerSecond"]] + [frame["participantFrames"][str(i)]["jungleMinionsKilled"]] + [frame["participantFrames"][str(i)]["minionsKilled"]]])
        
        line_part[id_line_match[str(i)]] = data
    
    
    
    for frame in json_data["info"]["frames"][1:]:
        for i in range(1, 11):
            data = np.array(list(frame["participantFrames"][str(i)]["championStats"].values()))
            data = data - np.array(line_part[id_line_match[str(i)]][0][:25])
            stats = np.array(list(frame["participantFrames"][str(i)]["damageStats"].values()) + [frame["participantFrames"][str(i)]["currentGold"]] + [frame["participantFrames"][str(i)]["goldPerSecond"]] + [frame["participantFrames"][str(i)]["jungleMinionsKilled"]] + [frame["participantFrames"][str(i)]["minionsKilled"]])
        
            data = np.concatenate((data, stats), axis = 0)
            line_part[id_line_match[str(i)]] = np.concatenate((line_part[id_line_match[str(i)]], np.array([data])), axis=0)
    line_part = np.array(line_part)
    
    return line_part, win_team

def print_graph(graph):
    i=0
    for vertex in graph:
        print(i, end=" ")
        for edge in graph[vertex]:
            if graph[vertex][edge] != 0:
                print(list(graph.keys()).index(edge), end=" ")
        i+=1
        print()

def write_graph(graph, name):
    f = open(name,'w')
    keys = list(graph.keys())
    
    n=0
    for g in graph:
        for i in graph[g]:
            if n%10000 == 0:
                print(i,n)
            n+=1
            f.write("%d %d\n"%(keys.index(g), keys.index(i)))
            
    f.close()

In [7]:
X = []
y = []
file_list = os.listdir(path_dir)
graph = {}
i=0

for file_name in file_list:
    if i%1000 == 0:
        print(i, "/", len(file_list))
    json_name = path_dir + '/' + file_name
    json_name_match = path_dir_match + '/' + file_name
    
    data, win = get_graph(json_name, json_name_match)
    
    if data.size:
        if len(data.shape)==3:
            #print_graph(game_G)
            X.append(data)
            y.append(win)
    #for g in graph:
    #    print(g, graph[g])
    i+=1
    

0 / 34287
{'1': 0, '2': 1, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '8': 7, '9': 8, '10': 6}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '10': 9}
{'1': 0, '2': 1, '3': 2, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'2': 1, '3': 2, '4': 3, '5': 0, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '7': 6, '8': 8, '9': 9, '10': 7}
{'1': 0, '2': 1, '3': 2, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
1000 / 34287
{'2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '8': 7, '9': 8, '10': 9}
11000 / 34287
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '7': 6, '8': 7, '9': 8, '10': 9}
{'2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '9': 8, '10': 9}
{'2': 1, '3': 2, '4': 3, 

{'2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 2, '3': 1, '4': 3, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
24000 / 34287
{'1': 0, '2': 1, '3': 2, '4': 3, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'2': 1, '3': 2, '4': 3, '5': 0, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '10': 9}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '8': 7, '9': 8, '10': 9}
{'1': 0, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '2': 1, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
{'1': 0, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
25000 / 34287
{'1': 0, '

In [8]:
lanes = ["top_", "mid_", "jug_", "adc_", "sup_"]
features = {'kills':[], 'deaths':[], 'assists':[], 'largestKillingSpree':[], 'longestTimeSpentLiving':[], 'totalDamageDealt':[], 
        'magicDamageDealt':[], 'physicalDamageDealt':[], 'trueDamageDealt':[], 'totalDamageDealtToChampions':[], 
        'magicDamageDealtToChampions':[], 'physicalDamageDealtToChampions':[], 'trueDamageDealtToChampions':[],
        'totalHeal':[], 'totalUnitsHealed':[], 'totalDamageTaken':[], 'goldEarned':[],
        'totalMinionsKilled':[], 'timeCCingOthers':[], 'wardsPlaced':[], 'wardsKilled':[]}
for f in features:
    print(f, end=", ")

kills, deaths, assists, largestKillingSpree, longestTimeSpentLiving, totalDamageDealt, magicDamageDealt, physicalDamageDealt, trueDamageDealt, totalDamageDealtToChampions, magicDamageDealtToChampions, physicalDamageDealtToChampions, trueDamageDealtToChampions, totalHeal, totalUnitsHealed, totalDamageTaken, goldEarned, totalMinionsKilled, timeCCingOthers, wardsPlaced, wardsKilled, 

In [10]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
max_len = max(l.shape[1] for l in X)
max_len

61

In [12]:
np.array(X[:2])
"""
for l in X:
    #if len(l.shape)!=3:
        print(len(l.shape))
        """

ValueError: could not broadcast input array from shape (10,21,41) into shape (10,)

In [13]:
X_np = X
"""
temp = np.pad(X[0], ((0, 0), (0, max_len-X[0].shape[1]), (0, 0)), 'constant')
temp = np.transpose(temp, (1, 0, 2))

temp = temp.reshape(max_len, 410)
X_np = np.array([temp])

print(X_np.shape)
"""
for i, l in enumerate(X):
    if i%1000 == 0:
        print(str(i),"/",len(X))
    temp = np.pad(l, ((0, 0), (0, max_len-l.shape[1]), (0, 0)), 'constant')
    
    temp = np.transpose(temp, (1, 0, 2))
    
    temp = temp.reshape(max_len, 410)
    
    X[i] =temp
    

0 / 33976
1000 / 33976
2000 / 33976
3000 / 33976
4000 / 33976
5000 / 33976
6000 / 33976
7000 / 33976
8000 / 33976
9000 / 33976
10000 / 33976
11000 / 33976
12000 / 33976
13000 / 33976
14000 / 33976
15000 / 33976
16000 / 33976
17000 / 33976
18000 / 33976
19000 / 33976
20000 / 33976
21000 / 33976
22000 / 33976
23000 / 33976
24000 / 33976
25000 / 33976
26000 / 33976
27000 / 33976
28000 / 33976
29000 / 33976
30000 / 33976
31000 / 33976
32000 / 33976
33000 / 33976


In [14]:
X_np = np.array(X_np)
X_np.shape

(33976, 61, 410)

In [15]:
y_np = np.array(y)
y_np.shape


(33976,)

In [10]:
if os.path.isfile("League_LSTM_twoteam.npy"):
    X_np = np.load("League_LSTM_twoteam.npy")
    y_np = np.load("League_LSTM_twoteam_value.npy")

In [11]:
X_np[0][2]

array([   0,    0,    6,    0,    0,   37,   18,    0,    0,    0,    0,
         24,   80,   25,    0,    0,    0,    0,    0,    0,    0,   26,
          0,    0,    0,    0,    0,    0,  402,   65,   80,  402,   65,
         80,    0,    0,    0,   63,    0,    0,    2,    0,    9,    8,
          0,    0,   37,    1,    0,    0,    0,    0,  -40,   68,   30,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       1525,    0,    0,  833,    0,  536, 2862,    0,  536,  504,    0,
          0,  160,    0,    5,    0,    0,   15,    0,    0,    0,   26,
         10,    0,    0,    0,    0, -249,   70,  111,    0,    0,    0,
         16,    0,    0,    0,  -46,    0,   16,    0,   53,    0,  116,
        869,   85,  245,  923,   85,  361,    0,    0,    0,   84,    0,
          0,    3,    0,    0,    6,    0,    0,   50,   20,    0,    0,
          0,    0,    0,    0,    7,    0,    0,    0,    0,   30,    0,
          0,  -79,    0,   13,    0,   18,    0,   

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size = 0.1, random_state = 0)

In [13]:
# check the shape of X_train and X_test
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(30578, 61, 410) (3398, 61, 410)
(30578,) (3398,)


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input, LSTM
max_len=61
model = Sequential()
model.add(Input(shape=(max_len, 410), dtype='float32'))

model.add(LSTM(410, activation = 'tanh', return_sequences=True, input_shape=(max_len, 410)))
model.add(LSTM(410, activation = 'tanh', input_shape=(max_len, 410)))

model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 61, 410)           1346440   
_________________________________________________________________
lstm_1 (LSTM)                (None, 410)               1346440   
_________________________________________________________________
dense (Dense)                (None, 1)                 411       
Total params: 2,693,291
Trainable params: 2,693,291
Non-trainable params: 0
_________________________________________________________________


In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
EPOCH = 100
BATCH_SIZE = 32
filename = 'checkpoint-epoch-{}-batch-{}-trial-001.h5'.format(EPOCH, BATCH_SIZE)
checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True,  # 가장 best 값만 저장합니다
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )

earlystopping = EarlyStopping(monitor='val_loss',  # 모니터 기준 설정 (val loss) 
                              patience=10,         # 10회 Epoch동안 개선되지 않는다면 종료
                             )

In [18]:
from tensorflow.debugging import set_log_device_placement
set_log_device_placement(True)

In [19]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2938639502707938834
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4992860160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15954425624295497143
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [20]:
from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.0001, momentum=0.9, clipvalue=5.0)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
#model.fit(X_train, y_train, batch_size = 32, epochs=100, verbose=1)
model.fit(X_train, y_train, epochs=EPOCH,  
                 batch_size=BATCH_SIZE, verbose=1, validation_split=0.2,   
                 callbacks=[checkpoint, earlystopping])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 24462 samples, validate on 6116 samples
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5

Epoch 1/100
24448/24462 [============================>.] - ETA: 0s - loss: 0.6928 - acc: 0.5182
Epoch 00001: val_loss improved from inf to 0.69228, saving model to checkpoint-epoch-100-batch-32-trial-001.h5
24462/24462 [==============================] - 135s 6ms/sample - loss: 0.6928 - acc: 0.5181 - val_loss: 0.6923 - val_acc: 0.5334
Epoch 2/100
24448/24462 [============================>.] - ETA: 0s - loss: 0.6922 - acc: 0.5174
Epoch 00002: val_loss improved from 0.69228 to 0.69173, saving model to checkpoint-epoch-100-batch-32-trial-001.h5
24462/24462 [==============================] - 135s 6ms/sample - loss: 0.6923 - acc: 0.5172 - val_loss: 0.6917 - val_acc: 0.5309
Epoch 3/100
24448/

24448/24462 [============================>.] - ETA: 0s - loss: 0.4863 - acc: 0.7482
Epoch 00024: val_loss did not improve from 0.47042
24462/24462 [==============================] - 118s 5ms/sample - loss: 0.4862 - acc: 0.7483 - val_loss: 0.4859 - val_acc: 0.7407
Epoch 25/100
24448/24462 [============================>.] - ETA: 0s - loss: 0.4640 - acc: 0.7608
Epoch 00025: val_loss did not improve from 0.47042
24462/24462 [==============================] - 117s 5ms/sample - loss: 0.4639 - acc: 0.7609 - val_loss: 0.4813 - val_acc: 0.7498
Epoch 26/100
24448/24462 [============================>.] - ETA: 0s - loss: 0.4777 - acc: 0.7517
Epoch 00026: val_loss did not improve from 0.47042
24462/24462 [==============================] - 118s 5ms/sample - loss: 0.4776 - acc: 0.7517 - val_loss: 0.4896 - val_acc: 0.7417
Epoch 27/100
24448/24462 [============================>.] - ETA: 0s - loss: 0.4752 - acc: 0.7597
Epoch 00027: val_loss did not improve from 0.47042
24462/24462 [=====================

In [ ]:
np.save("League_LSTM_twoteam", X_np)
np.save("League_LSTM_twoteam_value", y_np)


In [22]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

3398/3398 [==============================] - 8s 2ms/sample - loss: 0.4908 - acc: 0.7416

 테스트 정확도: 0.7416
